### Setup

In [1]:
### Imports
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os
import pandas as pd
from tqdm import tqdm
from numba import njit
from dieboldmariano import dm_test

In [2]:
def mse(y, y_hat):
    return np.mean((y - y_hat)**2)

def rmse_ratio(YT, forecasts1, forecasts2):
    mse1 = np.sqrt(mse(YT, forecasts1))
    mse2 = np.sqrt(mse(YT, forecasts2))
    return mse1/mse2

#### Data

In [3]:
# Load data from CSV
# data = pd.read_csv("data.csv")
data = pd.read_csv("data_extended.csv")


### ARMA

In [4]:
import warnings
warnings.filterwarnings("ignore")

### Fit the ARMA model
# Define a function to find the best parameters p in {0,1,...,12} and q in {0,1,2} for the ARMA model
def find_best_arma(YT, init=0):
    bic_smallest = float('inf')
    # for p in range(0, 13): # Officially we look over {0,1,...,12}
    for p in range(0, 11): # but we change it to {0,1,...,10} for faster computation
        for q in range(0, 3):
            arma_model = sm.tsa.arima.ARIMA(YT, order=(p,0,q))
            if init == 1:
                arma_model.initialize_approximate_diffuse()
            elif init == 2:
                arma_model.initialize_stationary()
            arma_fitted = arma_model.fit()
            bic_current = arma_fitted.bic
            if bic_current < bic_smallest:
                bic_smallest = bic_current
                best_model = arma_fitted
    return best_model

In [ ]:
col = 'UNRATE'
YT = data[col].values

test_begin = data.index[data['sasdate'] == '08/01/1999'][0]
test_end = data.index[data['sasdate'] == '7/1/2015'][0]

YT_train = YT[:test_begin]
YT_test = YT[test_begin:]

In [ ]:
# Fit ARMA model to the whole data (replication)
steps = 1
arma_fitted = find_best_arma(YT_train)
print("YT_train: ARMA(", arma_fitted.model_orders['ar'], ",", arma_fitted.model_orders['ma'], ")")

predictions_ARMA = np.concatenate([arma_fitted.predict(), arma_fitted.forecast(steps=1), np.zeros(len(YT_test)-1)])

for t in tqdm(range(len(YT_train)+1, len(YT))):
    if t == len(YT_train)+1 or t % steps == 0:
        arma_fitted = find_best_arma(YT[:t])
        p, q = (arma_fitted.model_orders['ar'], arma_fitted.model_orders['ma'])
        print(t-1)
        print("ARMA(", arma_fitted.model_orders['ar'], ",", arma_fitted.model_orders['ma'], ")")
    else: 
        arma_fitted = sm.tsa.arima.ARIMA(YT[:t], order=(p,0,q)).fit()
  
    predictions_ARMA[t] = arma_fitted.forecast(steps=1)[0]

In [ ]:
# Extend fitting the data to the extended data
col = 'UNRATE'
YT = np.array(data[col])
predictions_df = pd.read_csv(f'{col}_predictions.csv')
predictions_ARMA = np.append(np.array(predictions_df['ARMA']), np.zeros(len(YT)-len(predictions_df['ARMA']))) # For some magic reason, np.concatenate does not work here

for t in tqdm(range(test_end+1, len(YT))):
    if t == test_end+1 or t % steps == 0:
        arma_fitted = find_best_arma(YT[:t])
        p, q = (arma_fitted.model_orders['ar'], arma_fitted.model_orders['ma'])
        print(t-1)
        print("ARMA(", arma_fitted.model_orders['ar'], ",", arma_fitted.model_orders['ma'], ")")
    else: 
        arma_fitted = sm.tsa.arima.ARIMA(YT[:t], order=(p,0,q)).fit()
  
    predictions_ARMA[t] = arma_fitted.forecast(steps=1)[0]

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(YT, linewidth=0.3, color='blue', label='True values')
plt.plot(predictions_ARMA, linewidth=0.3, color='red', label='ARMA predictions')
plt.title('ARMA 1-step ahead')
plt.xlim(0, len(YT))
plt.axvline(x=len(YT_train), color='black', linestyle='--')
plt.legend()
plt.show()


# print MSE
print('in-sample MSE:', mse(YT_train[1:], predictions_ARMA[1:len(YT_train)]))
print('out-of-sample MSE:', mse(YT_test, predictions_ARMA[len(YT_train):]))

In [ ]:
# Save results to csv file
predictions_df = pd.DataFrame()
predictions_df['True values'] = YT
predictions_df['ARMA'] = predictions_ARMA

# predictions_df.to_csv('UNRATE_predictions.csv', index=False)
# predictions_df.to_csv('FEDFUNDS_predictions.csv', index=False)
# predictions_df.to_csv('INDPRO_predictions.csv', index=False)
# predictions_df.to_csv('PCEPI_predictions.csv', index=False)
# predictions_df.to_csv('RCON_predictions.csv', index=False)
# predictions_df.to_csv('AVGEARN_predictions.csv', index=False)
# predictions_df.to_csv('CUMFNS_predictions.csv', index=False)
# predictions_df.to_csv('PAYEMS_predictions.csv', index=False)

In [ ]:
# Load from csv file
col = 'FEDFUNDS'
predictions_df = pd.read_csv(f'{col}_predictions.csv')

YT = np.array(predictions_df['True values'])
YT_train = np.array(YT[:test_begin])
YT_test = np.array(YT[test_begin:])

predictions_ARMA = np.array(predictions_df['ARMA']) 

### Forecast adjustment

In [5]:
### Match-to-level
# Define the distance function
@njit
def dist(Y1, Y2):
    # if len(Y1) != len(Y2): # Check 
    #     return TypeError
    k = len(Y1)
    sum = 0
    for i in range(0, k):
        Wi = 1/(k-i)
        sum += Wi * (Y1[i] - Y2[i])**2
    return sum

@njit
def find_similar_periods(YT, t, m, k):
    '''
    Input:
        YT: univariate series of data
        t: current index
        k: length of interval to be matched
        m: number of matching blocks to be returned
    Output:
        the m best matching blocks of length k, return tuple:(distance, block, end-index of block)
    '''
    num_blocks = t - k + 1
    distances = np.empty((num_blocks, 3), dtype=np.float64)
    XT = YT[t-k+1:t+1]

    for i in range(num_blocks):
        current_block = YT[i:i+k]
        distance = dist(XT, current_block)
        distances[i, 0] = distance
        distances[i, 1] = i
        distances[i, 2] = i + k

    # Sort based on distance
    sorted_indices = np.argsort(distances[:, 0])
    sorted_distances = distances[sorted_indices]

    # Retrieve the top m blocks
    top_m_blocks = sorted_distances[:m]

    # Extract blocks with their corresponding distance and end index
    result = [(top_m_blocks[i, 0], YT[int(top_m_blocks[i, 1]):int(top_m_blocks[i, 1])+k], int(top_m_blocks[i, 2])) for i in range(m)]
    
    return result

### Forecast Adjustment
# Define function to predict single values
def predict(YT, t, best_fits, ARMA_predictions, h=0):
    '''
    Input:
        YT: ndarray, univariate series of data
        t: int, current index
        best_fits: (m x 3) ndarray, the m best matching blocks. for each m: (distances, block, end-index)
        arma_predictions: ndarray, predictions from ARMA model
        h: int, the number of steps ahead to forecast (if h>0)
    Output:
        forecast: float, the adjusted forecast for index t
    '''
    m = len(best_fits)
    sum = 0
    for fit in best_fits:
        matching_index = fit[2]
        sum += YT[matching_index+h] - ARMA_predictions[matching_index+h]
    return sum/m + ARMA_predictions[t+h]


# Find the best parameters
def optimal_m_k(YT, ARMA_predictions, t1, T):
    mse_dict = {} # Dictionary that maps (m,k) to corresponding MSE

    # Test different parameters
    for m in range(2,80, 6):
        for k in range(2, 80, 6):
            sum_squared = 0
            count = 0
            for t in range(t1, T-1):
                best_fits = find_similar_periods(YT, t, m, k)
                forecast = predict(YT, t, best_fits, ARMA_predictions, 1)
                sum_squared += (YT[t+1] - forecast)**2
                count += 1
            
            mse_current = sum_squared / count
            mse_dict[(m, k)] = mse_current

    # Select best 1/4th of the parameters
    sorted_mse_dict = sorted(mse_dict.items(), key=lambda x: x[1])
    best_params = sorted_mse_dict[:len(sorted_mse_dict)//4]
    # Take the average of the best parameters
    m = int(np.mean([param[0][0] for param in best_params]))
    k = int(np.mean([param[0][1] for param in best_params]))
    return m, k


In [ ]:
data = pd.read_csv('data_extended.csv')
col = 'FEDFUNDS'
print(f'Now computing for {col}')

YT = np.array(data[col])

t1 = data.index[data['sasdate'] == '01/01/1975'][0]
test_begin = data.index[data['sasdate'] == '08/01/1999'][0]
test_end = data.index[data['sasdate'] == '7/1/2015'][0]
recession_start = data.index[data['sasdate'] == '12/01/2007'][0]
recession_end = data.index[data['sasdate'] == '06/01/2009'][0]
covid_start = data.index[data['sasdate'] == '2/1/2020'][0]
covid_end = data.index[data['sasdate'] == '6/1/2021'][0]

# Fit ARMA model to the whole data
arma_fitted = find_best_arma(YT[:test_begin])
predictions_ARMA = np.concatenate([arma_fitted.predict(), arma_fitted.forecast(steps=1), np.zeros(len(YT)-test_begin-1)])
for t in tqdm(range(test_begin+1, len(YT))):
    arma_fitted = find_best_arma(YT[:t])
    predictions_ARMA[t] = arma_fitted.forecast(steps=1)[0]
    print(f"\tp={arma_fitted.model_orders['ar']}, q={arma_fitted.model_orders['ma']}")

print(f"\tFinished fitting ARMA model for {col}")

# Fit NN model to the whole data
m, k = optimal_m_k(YT, predictions_ARMA, t1, test_begin)
predictions_NN = np.zeros(len(YT))
for t in range(t1, test_begin+1): # t1 - end of in-sample period
    best_fits = find_similar_periods(YT, t, m, k)
    predictions_NN[t] =  predict(YT, t, best_fits, predictions_ARMA)
for t in tqdm(range(test_begin+1, len(YT))): # out-of-sample period
    m, k = optimal_m_k(YT, predictions_ARMA, t1, t)
    best_fits = find_similar_periods(YT, t-1, m, k)
    predictions_NN[t] = predict(YT, t-1, best_fits, predictions_ARMA, h=1)

print(f"\tFinished fitting NN model for {col}")

# Plot the results
plt.figure(figsize=(20,6))
plt.plot(YT, linewidth=0.8, color='blue', label='True values')
plt.plot(predictions_ARMA, linewidth=0.8, color='red', label='ARMA predictions')
plt.plot(predictions_NN, linewidth=0.8, color='green', label='NN predictions')
plt.title(f'ARMA and NN forecasts for {col}')
plt.xlim(0, len(YT))
plt.axvline(x=test_begin, color='black', linestyle='--')
plt.axvline(x=test_end, color='black', linestyle='--', linewidth=0.5)
plt.axvline(x=recession_start, color='pink', linewidth=0.5)
plt.axvline(x=covid_start, color='pink', linewidth=0.5)
plt.legend()
plt.show()

# Compute MSE ratios

# Table 1: out-of-sample - 2015
ratio = rmse_ratio(YT[test_begin:test_end], predictions_NN[test_begin:test_end], predictions_ARMA[test_begin:test_end])
print('\tTable 1 RMSE ratio:', round(ratio, 4))
# Great recession: 12-2007 to 06-2009
ratio = rmse_ratio(YT[recession_start:recession_end+1], predictions_NN[recession_start:recession_end+1], predictions_ARMA[recession_start:recession_end+1])
print('\tGreat recession RMSE ratio:', round(ratio, 4))
# Ex-great recession: 07-2009 to 07-2015 
ratio = rmse_ratio(YT[recession_end+1:test_end], predictions_NN[recession_end+1:test_end], predictions_ARMA[recession_end+1:test_end])   
print('\tEx-great recession RMSE ratio:', round(ratio, 4)) 
# Covid recession: 2-2020 to 6-2021
ratio = rmse_ratio(YT[covid_start:covid_end+1], predictions_NN[covid_start:covid_end+1], predictions_ARMA[covid_start:covid_end+1])
print('\tCovid recession RMSE ratio:', round(ratio, 4))
# Ex-covid recession: 7-2021 to 03-2024
ratio = rmse_ratio(YT[covid_end+1:], predictions_NN[covid_end+1:], predictions_ARMA[covid_end+1:])
print('\tEx-covid recession RMSE ratio:', round(ratio, 4))

# # Save results to csv file
# predictions_df = pd.DataFrame()
# predictions_df['True values'] = YT
# predictions_df['ARMA'] = predictions_ARMA
# predictions_df['NN'] = predictions_NN

# if f"{col}_predictions.csv"  in os.listdir():
#     predictions_df_old = pd.read_csv(f'{col}_predictions.csv')
#     predictions_df['ARMA'] = predictions_df_old['ARMA']
#     predictions_df['NN'] = predictions_df_old['NN']

# predictions_df.to_csv(f'{col}_predictions.csv', index=False)


In [ ]:
def print_stuff(actual, pred1, pred2, title):
    ratio = rmse_ratio(actual, pred1, pred2)
    _, p_value = dm_test(actual, pred1, pred2, one_sided=True)
    print(f'\t{title} RMSE ratio:\t\t {round(ratio, 4)} ({round(p_value, 2)})')

In [ ]:
for col in data.columns:
    if col == 'sasdate':
        continue
    if f"{col}_predictions.csv" in os.listdir():
        predictions_df = pd.read_csv(f"{col}_predictions.csv")
        YT = np.array(predictions_df['True values'])
        predictions_ARMA = np.array(predictions_df['ARMA'])
        predictions_NN = np.array(predictions_df['NN'])
        # print(var_dict[col])
        print(col)
    else:
        print(f"Skipping {col}")
        continue

    # Compute RMSE ratios
    # Table 1: out-of-sample - 2015
    # print_stuff(YT[test_begin:test_end], predictions_NN[test_begin:test_end], predictions_ARMA[test_begin:test_end], '')
    # # Great recession: 12-2007 to 06-2009
    print_stuff(YT[recession_start:recession_end+1], predictions_NN[recession_start:recession_end+1], predictions_ARMA[recession_start:recession_end+1], 'Great recession')
    # # Ex-great recession: 07-2009 to 07-2015 
    print_stuff(YT[recession_end+1:test_end], predictions_NN[recession_end+1:test_end], predictions_ARMA[recession_end+1:test_end], 'Ex-great recession') 
    # # Covid recession: 2-2020 to 6-2021
    print_stuff(YT[covid_start:covid_end+1], predictions_NN[covid_start:covid_end+1], predictions_ARMA[covid_start:covid_end+1], 'Covid recession')
    # # Ex-covid recession: 7-2021 to 03-2024
    print_stuff(YT[covid_end+1:], predictions_NN[covid_end+1:], predictions_ARMA[covid_end+1:], 'Ex-covid recession')

### Exponential Smoothing:

Y_{T+1} = (1 − 𝛾) / (1 − 𝛾^T) sum (𝛾^(T-t)Y_t)


In [9]:
@njit
def predict_ES(YT, t, gamma, h=0, t0=0):
    return (1-gamma) / (1-gamma**(t-t0)) * sum([gamma**(t-i) * YT[i+h] for i in range(t0, t)])

def optimal_gamma(YT, t1, T, t0=0):
    rmse_dict = {} # Dictionary that maps (m,k) to corresponding MSE

    # Test different parameters
    for gamma in np.arange(0.01, 1, 0.01):
        sum_squared = 0
        count = 0
        for t in range(t1, T-1):
            forecast = predict_ES(YT, t, gamma, 1, t0)
            sum_squared += (YT[t+1] - forecast)**2
            count += 1
        
        rmse_current = np.sqrt(sum_squared / count)
        rmse_dict[gamma] = rmse_current

    # Select best 1/4th of the parameters
    sorted_rmse_dict = sorted(rmse_dict.items(), key=lambda x: x[1])
    best_params = sorted_rmse_dict[:len(sorted_rmse_dict)//4]
    # Take the average of the best parameters
    return np.mean([param[0] for param in best_params])

In [18]:
data = pd.read_csv('data_extended.csv')
for col in data.columns:
    if col == 'sasdate':
        continue
    print(f'Now computing for {col}')
    predictions_df = pd.read_csv(f'{col}_predictions.csv')
    YT = np.array(predictions_df['True values'])
    predictions_ARMA = np.array(predictions_df['ARMA'])
    predictions_NN = np.array(predictions_df['NN'])

    t1 = data.index[data['sasdate'] == '01/01/1975'][0]
    test_begin = data.index[data['sasdate'] == '08/01/1999'][0]
    test_end = data.index[data['sasdate'] == '7/1/2015'][0]
    recession_start = data.index[data['sasdate'] == '12/01/2007'][0]
    recession_end = data.index[data['sasdate'] == '06/01/2009'][0]
    covid_start = data.index[data['sasdate'] == '2/1/2020'][0]
    covid_end = data.index[data['sasdate'] == '6/1/2021'][0]

    # Fit ES model to the whole data

    # predictions_ES = np.zeros(len(YT))
    # gammas = np.zeros(len(YT))
    # for t in tqdm(range(test_begin, len(YT))):
    #     gamma = optimal_gamma(YT, t1, t)
    #     gammas[t] = gamma
    #     predictions_ES[t] = predict_ES(YT, t-1, gamma, 1)

    # predictions_df['ES'] = predictions_ES
    # predictions_df.to_csv(f'{col}_predictions.csv', index=False)
    # print(gammas[test_begin:])
    predictions_ES = np.array(predictions_df['ES'])



    # Plot the results
    # plt.figure(figsize=(20,6))
    # plt.plot(YT, linewidth=0.8, color='blue', label='True values')
    # plt.plot(predictions_ES, linewidth=0.8, color='purple', label='ARMA predictions')
    # plt.xlim(0, len(YT))
    # plt.axvline(x=test_begin, color='black', linestyle='--')
    # plt.axvline(x=recession_start, color='pink', linewidth=0.5)
    # plt.axvline(x=covid_start, color='pink', linewidth=0.5)
    # plt.legend()
    # plt.show()

    # Compute RMSE ratios
    # Table 1: out-of-sample - 2015
    print_stuff(YT[test_begin:], predictions_NN[test_begin:], predictions_ES[test_begin:], 'Table 1\t\t\t')
    # # Great recession: 12-2007 to 06-2009
    print_stuff(YT[recession_start:recession_end+1], predictions_NN[recession_start:recession_end+1], predictions_ES[recession_start:recession_end+1], 'Great recession')
    # # Ex-great recession: 07-2009 to 07-2015 
    print_stuff(YT[recession_end+1:test_end], predictions_NN[recession_end+1:test_end], predictions_ES[recession_end+1:test_end], 'Ex-great recession') 
    if col != 'RCON':
        pass
        # # Covid recession: 2-2020 to 6-2021
        print_stuff(YT[covid_start:covid_end+1], predictions_NN[covid_start:covid_end+1], predictions_ES[covid_start:covid_end+1], 'Covid recession')
        # # Ex-covid recession: 7-2021 to 03-2024
        print_stuff(YT[covid_end+1:], predictions_NN[covid_end+1:], predictions_ES[covid_end+1:], 'Ex-covid recession')



Now computing for UNRATE
	Great recession RMSE ratio:		 0.9482 (0.13)
	Ex-great recession RMSE ratio:		 1.0171 (0.67)
	Covid recession RMSE ratio:		 1.0629 (0.81)
	Ex-covid recession RMSE ratio:		 1.1034 (0.89)
Now computing for INDPRO
	Great recession RMSE ratio:		 1.0049 (0.59)
	Ex-great recession RMSE ratio:		 0.971 (0.25)
	Covid recession RMSE ratio:		 0.9826 (0.06)
	Ex-covid recession RMSE ratio:		 0.9486 (0.11)
Now computing for FEDFUNDS
	Great recession RMSE ratio:		 0.7903 (0.05)
	Ex-great recession RMSE ratio:		 1.806 (1.0)
	Covid recession RMSE ratio:		 0.9605 (0.33)
	Ex-covid recession RMSE ratio:		 0.7605 (0.01)
Now computing for S&P 500
	Great recession RMSE ratio:		 0.962 (0.08)
	Ex-great recession RMSE ratio:		 0.9941 (0.41)
	Covid recession RMSE ratio:		 1.0332 (0.94)
	Ex-covid recession RMSE ratio:		 0.9907 (0.39)
Now computing for PAYEMS
	Great recession RMSE ratio:		 0.8895 (0.04)
	Ex-great recession RMSE ratio:		 1.0419 (0.8)
	Covid recession RMSE ratio:		 1.4667 (0

In [16]:
### Fixing NaN values on Busloans and REALLN 
col = 'REALLN'  
print(f'Now computing for {col}')
predictions_df = pd.read_csv(f'{col}_predictions.csv')
YT = np.array(predictions_df['True values'])
predictions_ARMA = np.array(predictions_df['ARMA'])
predictions_NN = np.array(predictions_df['NN'])

t1 = data.index[data['sasdate'] == '01/01/1975'][0]
test_begin = data.index[data['sasdate'] == '08/01/1999'][0]
test_end = data.index[data['sasdate'] == '7/1/2015'][0]
recession_start = data.index[data['sasdate'] == '12/01/2007'][0]
recession_end = data.index[data['sasdate'] == '06/01/2009'][0]
covid_start = data.index[data['sasdate'] == '2/1/2020'][0]
covid_end = data.index[data['sasdate'] == '6/1/2021'][0]

if col == 'BUSLOANS' or col == 'REALLN':
    t0 = data.index[data['sasdate'] == '02/01/1973'][0]
else:
    t0 = 0

# Fit ES model to the whole data
predictions_ES = np.zeros(len(YT))
gammas = np.zeros(len(YT))
for t in tqdm(range(test_begin, len(YT))):
    gamma = optimal_gamma(YT, t1, t, t0)
    gammas[t] = gamma
    predictions_ES[t] = predict_ES(YT, t-1, gamma, 1, t0)

predictions_df['ES'] = predictions_ES
predictions_df.to_csv(f'{col}_predictions.csv', index=False)
# print(gammas[test_begin:])
predictions_ES = np.array(predictions_df['ES'])



# Plot the results
# plt.figure(figsize=(20,6))
# plt.plot(YT, linewidth=0.8, color='blue', label='True values')
# plt.plot(predictions_ES, linewidth=0.8, color='purple', label='ARMA predictions')
# plt.xlim(0, len(YT))
# plt.axvline(x=test_begin, color='black', linestyle='--')
# plt.axvline(x=recession_start, color='pink', linewidth=0.5)
# plt.axvline(x=covid_start, color='pink', linewidth=0.5)
# plt.legend()
# plt.show()

# Compute RMSE ratios
# Table 1: out-of-sample - 2015
# print_stuff(YT[test_begin:test_end], predictions_NN[test_begin:test_end], predictions_ES[test_begin:test_end], 'Table 1\t\t\t')
# # Great recession: 12-2007 to 06-2009
print_stuff(YT[recession_start:recession_end+1], predictions_NN[recession_start:recession_end+1], predictions_ES[recession_start:recession_end+1], 'Great recession')
# # Ex-great recession: 07-2009 to 07-2015 
print_stuff(YT[recession_end+1:test_end], predictions_NN[recession_end+1:test_end], predictions_ES[recession_end+1:test_end], 'Ex-great recession') 
if col != 'RCON':
    # # Covid recession: 2-2020 to 6-2021
    print_stuff(YT[covid_start:covid_end+1], predictions_NN[covid_start:covid_end+1], predictions_ES[covid_start:covid_end+1], 'Covid recession')
    # # Ex-covid recession: 7-2021 to 03-2024
    print_stuff(YT[covid_end+1:], predictions_NN[covid_end+1:], predictions_ES[covid_end+1:], 'Ex-covid recession')

Now computing for REALLN


  0%|          | 0/296 [00:00<?, ?it/s]

100%|██████████| 296/296 [02:37<00:00,  1.88it/s]

	Great recession RMSE ratio:		 1.0526 (0.67)
	Ex-great recession RMSE ratio:		 1.0844 (0.88)
	Covid recession RMSE ratio:		 1.1409 (0.83)
	Ex-covid recession RMSE ratio:		 0.8083 (0.03)


### NN with DTW

In [105]:
import warnings
warnings.filterwarnings("ignore")
# Define a function to find the best parameters p in {0,1,...,10} and q in {0,1,2} for the ARMA model
def find_best_arma(YT):
    bic_smallest = float('inf')
    for p in range(0, 11):
        for q in range(0, 3):
            arma_model = sm.tsa.arima.ARIMA(YT, order=(p,0,q))
            arma_fitted = arma_model.fit()
            bic_current = arma_fitted.bic
            if bic_current < bic_smallest:
                bic_smallest = bic_current
                best_model = arma_fitted
    return best_model

### Match-to-level
# Define the distance function
# @njit # Speeeeeed
# def dist(Y1, Y2):
#     # if len(Y1) != len(Y2): # Check 
#     #     return TypeError
#     k = len(Y1)
#     sum = 0
#     for i in range(0, k):
#         Wi = 1/(k-i)
#         sum += Wi * (Y1[i] - Y2[i])**2
#     return sum

### DTW
@njit
def dtw_distance(series1, series2):
    n, m = len(series1), len(series2)
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            # cost = (series1[i - 1] - series2[j - 1])**2 # DTW
            cost = abs(series1[i - 1] - series2[j - 1]) # DTW2
            dtw_matrix[i, j] = cost + min(dtw_matrix[i - 1, j],    # insertion
                                          dtw_matrix[i, j - 1],    # deletion
                                          dtw_matrix[i - 1, j - 1])  # match

    return dtw_matrix[n, m]


@njit # Speeeed
# Define the function to find the m best matching blocks of length k
def find_similar_periods(YT, t, m, k):
    '''
    Input:
        YT: univariate series of data
        t: current index
        k: length of interval to be matched
        m: number of matching blocks to be returned
    Output:
        the m best matching blocks of length k, return tuple:(distance, block, end-index of block)
    '''
    num_blocks = t - k + 1
    distances = np.empty((num_blocks, 3), dtype=np.float64)
    XT = YT[t-k+1:t+1]

    for i in range(num_blocks):
        current_block = YT[i:i+k]
        distance = dtw_distance(XT, current_block)
        distances[i, 0] = distance
        distances[i, 1] = i
        distances[i, 2] = i + k

    # Sort based on distance
    sorted_indices = np.argsort(distances[:, 0])
    sorted_distances = distances[sorted_indices]
    # Retrieve the top m blocks
    top_m_blocks = sorted_distances[:m]
    # Extract blocks with their corresponding distance and end index
    result = [(top_m_blocks[i, 0], YT[int(top_m_blocks[i, 1]):int(top_m_blocks[i, 1])+k], int(top_m_blocks[i, 2])) for i in range(m)]
    
    return result

### Forecast Adjustment
# Define function to predict single values
def predict(YT, t, best_fits, ARMA_predictions, h=0):
    '''
    Input:
        YT: ndarray, univariate series of data
        t: int, current index
        best_fits: (m x 3) ndarray, the m best matching blocks. for each m: (distances, block, end-index)
        arma_predictions: ndarray, predictions from ARMA model
        h: int, the number of steps ahead to forecast (if h>0)
    Output:
        forecast: float, the adjusted forecast for index t
    '''
    m = len(best_fits)
    sum = 0
    for fit in best_fits:
        matching_index = fit[2]
        sum += YT[matching_index+h] - ARMA_predictions[matching_index+h]
    return sum/m + ARMA_predictions[t+h]


# Find the best parameters
def optimal_m_k(YT, ARMA_predictions, t1, T):
    mse_dict = {} # Dictionary that maps (m,k) to corresponding MSE

    # Test different parameters
    for m in range(2,80, 6):
        for k in range(2, 80, 6):
            sum_squared = 0
            count = 0
            for t in range(t1, T-1):
                best_fits = find_similar_periods(YT, t, m, k)
                forecast = predict(YT, t, best_fits, ARMA_predictions, 1)
                sum_squared += (YT[t+1] - forecast)**2
                count += 1
            
            mse_current = sum_squared / count
            mse_dict[(m, k)] = mse_current

    # Select best 1/4th of the parameters
    sorted_mse_dict = sorted(mse_dict.items(), key=lambda x: x[1])
    best_params = sorted_mse_dict[:len(sorted_mse_dict)//4]
    # Take the average of the best parameters
    m = int(np.mean([param[0][0] for param in best_params]))
    k = int(np.mean([param[0][1] for param in best_params]))
    return m, k

In [ ]:
for col in data.columns:
    if col == 'sasdate':
        continue
    print(f'Now computing for {col}')
    predictions_df = pd.read_csv(f'{col}_predictions.csv')
    YT = np.array(predictions_df['True values'])
    predictions_ARMA = np.array(predictions_df['ARMA'])
    predictions_NN = np.array(predictions_df['NN'])

    t1 = data.index[data['sasdate'] == '01/01/1975'][0]
    test_begin = data.index[data['sasdate'] == '08/01/1999'][0]
    test_end = data.index[data['sasdate'] == '7/1/2015'][0]
    recession_start = data.index[data['sasdate'] == '12/01/2007'][0]
    recession_end = data.index[data['sasdate'] == '06/01/2009'][0]
    covid_start = data.index[data['sasdate'] == '2/1/2020'][0]
    covid_end = data.index[data['sasdate'] == '6/1/2021'][0]

    ### Fit NN_DTW model to the whole data
    # Re-select m and k every 20 iterations during normal times and every iteration in crises
    jumps = 20
    recompute_list = [*range(test_begin, recession_start, jumps), *range(recession_start, recession_end), *range(recession_end, covid_start, jumps), *range(covid_start, covid_end), *range(covid_end, len(YT), jumps)]

    predictions_NN_DTW = np.zeros(len(YT))
    for t in tqdm(range(test_begin, len(YT))):
        if t in recompute_list:
            m, k = optimal_m_k(YT, predictions_ARMA, t1, t)
            # print(f'm: {m}, k: {k}, on index {t}')
        best_fits = find_similar_periods(YT, t-1, m, k)
        predictions_NN_DTW[t] = predict(YT, t-1, best_fits, predictions_ARMA, h=1)

    predictions_df['NN_DTW2'] = predictions_NN_DTW # 2 for absolute local distance function 
    predictions_df.to_csv(f'{col}_predictions.csv', index=False)

In [12]:
### Printing results
var_dict = {
    'UNRATE' : 'Unemployment Rate',
    'INDPRO' : 'Industrial Production',
    'PCEPI' : 'Inflation',
    'FEDFUNDS' : 'Federal Funds Rate',
    'S&P 500' : 'S&P 500',
    'RCON' : 'Personal Consumption',
    'PAYEMS' : 'Payroll Employment',
    'RPI' : 'Personal Income',
    'AVGEARN' : 'Average Hourly Earnings',
    'HOUST' : 'Housing Starts',
    'CUMFNS' : 'Capacity Utilization',
    'REALLN' : 'Real Estate Loans',
    'BUSLOANS' : 'C&I Loans'
}

t1 = data.index[data['sasdate'] == '01/01/1975'][0]
test_begin = data.index[data['sasdate'] == '08/01/1999'][0]
test_end = data.index[data['sasdate'] == '7/1/2015'][0]
recession_start = data.index[data['sasdate'] == '12/01/2007'][0]
recession_end = data.index[data['sasdate'] == '06/01/2009'][0]
covid_start = data.index[data['sasdate'] == '2/1/2020'][0]
covid_end = data.index[data['sasdate'] == '6/1/2021'][0]

for col in data.columns:
    if col == 'sasdate':
        continue
    print(var_dict[col])

    predictions_df = pd.read_csv(f"{col}_predictions.csv")
    YT = np.array(predictions_df['True values'])
    predictions_ARMA = np.array(predictions_df['ARMA'])
    predictions_NN = np.array(predictions_df['NN'])
    predictions_ES = np.array(predictions_df['ES'])
    predictions_NN_DTW = np.array(predictions_df['NN_DTW']) # squared local distance function
    predictions_NN_DTW2 = np.array(predictions_df['NN_DTW2']) # absolute local distance function

    # Compute RMSE ratios
    # extended out-of-sample
    # print_stuff(YT[test_begin:], predictions_NN_DTW2[test_begin:], predictions_ARMA[test_begin:], predictions_NN[test_begin:], 'Table 1')
    # # Great recession: 12-2007 to 06-2009
    print_stuff(YT[recession_start:recession_end+1], predictions_NN_DTW[recession_start:recession_end+1], predictions_ARMA[recession_start:recession_end+1], predictions_NN[recession_start:recession_end+1], 'Great recession')
    # # Ex-great recession: 07-2009 to 07-2015 
    print_stuff(YT[recession_end+1:test_end], predictions_NN_DTW2[recession_end+1:test_end], predictions_ARMA[recession_end+1:test_end], predictions_NN[recession_end+1:test_end], 'Ex-great recession') 
    if col != 'RCON':
        # # Covid recession: 2-2020 to 6-2021
        # print_stuff(YT[covid_start:covid_end+1], predictions_NN_DTW[covid_start:covid_end+1], predictions_ARMA[covid_start:covid_end+1], predictions_NN[covid_start:covid_end+1], 'Covid recession')
        # # Ex-covid recession: 7-2021 to 03-2024
        # print_stuff(YT[covid_end+1:], predictions_NN_DTW[covid_end+1:], predictions_ARMA[covid_end+1:], predictions_NN[covid_end+1:], 'Ex-covid recession')
        pass

Unemployment Rate
	Great recession DTW RMSE ratio:		 0.8894
	Great recession Euclidan RMSE ratio:		 0.9289
	Great recession DM test p-value:		 0.31
	Ex-great recession DTW RMSE ratio:		 0.9585
	Ex-great recession Euclidan RMSE ratio:		 1.008
	Ex-great recession DM test p-value:		 0.06
Industrial Production
	Great recession DTW RMSE ratio:		 0.7625
	Great recession Euclidan RMSE ratio:		 0.9375
	Great recession DM test p-value:		 0.03
	Ex-great recession DTW RMSE ratio:		 0.9688
	Ex-great recession Euclidan RMSE ratio:		 0.9715
	Ex-great recession DM test p-value:		 0.46
Federal Funds Rate
	Great recession DTW RMSE ratio:		 0.8197
	Great recession Euclidan RMSE ratio:		 0.8605
	Great recession DM test p-value:		 0.29
	Ex-great recession DTW RMSE ratio:		 1.4175
	Ex-great recession Euclidan RMSE ratio:		 1.6419
	Ex-great recession DM test p-value:		 0.09
S&P 500
	Great recession DTW RMSE ratio:		 0.9118
	Great recession Euclidan RMSE ratio:		 0.9657
	Great recession DM test p-value:		 0.

### Old code

In [ ]:
# Recreate the AR MA coefficients plot from the online appendix
data = pd.read_csv('data.csv')
fig, axs = plt.subplots(len(data.columns)-1, 2, figsize=(20, 20))
i=0

for col in data.columns:
    if col == 'sasdate':
        continue
    print(col)

    YT = np.array(data[col])
    YT_train = np.array(YT[:487])
    YT_test = np.array(YT[487:])


    p = np.full(len(YT_test), np.nan)
    q = np.full(len(YT_test), np.nan)
    for t in tqdm(range(0, len(YT_test), 6)):
        # print(data.iloc[len(YT_train) + t]['sasdate'])
        best_ARMA = find_best_arma(YT[:len(YT_train) + t])
        p[t] = best_ARMA.model_orders['ar']
        q[t] = best_ARMA.model_orders['ma']

    # Interpolate
    p = pd.Series(p).interpolate()
    q = pd.Series(q).interpolate()

    # set indices to the sasdate
    p.index = data.index[len(YT_train):]
    q.index = data['sasdate'][len(YT_train):]

    # Plot the results in a subplot
    axs[i, 0].plot(pd.to_datetime(np.array(data.iloc[len(YT_train):]['sasdate'])), p)
    axs[i, 0].set_title(f"AR coefficients {col}")
    # axs[i, 0].tick_params(axis='x', labelrotation=45)

    axs[i, 1].plot(pd.to_datetime(np.array(data.iloc[len(YT_train):]['sasdate'])), q)
    axs[i, 1].set_title(f"MA coefficients {col}")
    # axs[i, 1].set_xticks(rotation=45)
    i+=1

In [ ]:
### PLots for Illustrative Simulation
col = 'SIM'

predictions_df = pd.read_csv(f'{col}_predictions.csv')
YT = np.array(predictions_df['True values'])
predictions_ARMA = np.array(predictions_df['ARMA'])
predictions_NN = np.array(predictions_df['NN'])
test_begin = 375
crisis_begin = 615
crisis_end = 715

predictions_NN[:test_begin] = np.zeros(test_begin)

# Compute RMSE ratio
ratio = rmse_ratio(YT[test_begin:], predictions_NN[test_begin:], predictions_ARMA[test_begin:])
print(f'RMSE ratio for {col}:', round(ratio, 4))
ratio = rmse_ratio(YT[crisis_begin:crisis_end], predictions_NN[crisis_begin:crisis_end], predictions_ARMA[crisis_begin:crisis_end])
print('RMSE ratio crisis:', round(ratio, 4))

# Compute the RMSE values
print('RMSE ARMA in-sample:', np.sqrt(mse(YT[:test_begin], predictions_ARMA[:test_begin])))
print('RMSE ARMA out-of-sample:', np.sqrt(mse(YT[test_begin:], predictions_ARMA[test_begin:])))
print('RMSE NN out-of-sample:', np.sqrt(mse(YT[test_begin:], predictions_NN[test_begin:])))


# Plot the results
plt.figure(figsize=(20,6))
plt.plot(YT, linewidth=0.8, color='blue', label='True values')
# plt.plot(predictions_ARMA, linewidth=0.8, color='red', label='ARMA predictions')
plt.plot(predictions_NN, linewidth=0.8, color='green', label='NN predictions')
# plt.title(f'Generated data')
# plt.title(f'Fitted ARMA model')
plt.title(f'Fitted NN model')
plt.xlim(0, len(YT))
plt.axvline(x=test_begin, color='black', linestyle='--')
plt.axvspan(crisis_begin, crisis_end, color='lightgrey', alpha=0.5, lw=2, label='Crisis Period')
plt.legend()
plt.show()